Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [3]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [4]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [5]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [6]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

## Task 1
- Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [12]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [15]:
train_set = df[df['Date'].dt.year <= 2016]
validation_set = df[df['Date'].dt.year == 2017]
test_set = train_set = df[df['Date'].dt.year >= 2018]

## Task 2
- Begin with baselines for classification.

In [16]:
df.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
       'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
       'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
       'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
       'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso',
       'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini',
       'Great'],
      dtype='object')

In [17]:
target = 'Great'
y_train = train_set[target]
y_train.value_counts(normalize=True)


True     0.578947
False    0.421053
Name: Great, dtype: float64

In [94]:
def split_x_y(df):
    target = 'Great'
    x = df.drop(target, axis = 1)
    y = df[target]
    return {'x':x, 'y':y}

In [95]:
train = split_x_y(train_set)
val = split_x_y(validation_set)
test = split_x_y(test_set)

In [19]:
train_set.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,...,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
77,California,2026-04-25,NaN,NaN,NaN,8.00,4.0,NaN,NaN,21.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
386,California,2018-01-02,NaN,NaN,NaN,7.25,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
387,Other,2018-01-09,4.5,3.8,NaN,4.19,3.0,NaN,NaN,NaN,...,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,False
388,California,2018-01-12,3.5,4.3,NaN,7.00,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
389,Other,2018-01-12,NaN,NaN,NaN,8.50,4.0,NaN,NaN,21.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [22]:
from sklearn.metrics import accuracy_score

y_val = validation_set[target]
y_pred = [True] * len(y_val)
accuracy_score(y_val, y_pred)

0.4470588235294118

In [81]:
X_train = train_set.drop(target, axis = 1)

In [25]:
X_train.isna().sum()

Burrito            0
Date               0
Yelp              35
Google            35
Chips             37
Cost               0
Hunger             0
Mass (g)          38
Density (g/mL)    38
Length             4
Circum             5
Volume             5
Tortilla           0
Temp               0
Meat               2
Fillings           1
Meat:filling       1
Uniformity         0
Salsa              3
Synergy            0
Wrap               1
Unreliable        35
NonSD             38
Beef              34
Pico              31
Guac              30
Cheese            33
Fries             35
Sour cream        35
Pork              34
Chicken           37
Shrimp            38
Fish              38
Rice              37
Beans             35
Lettuce           38
Tomato            38
Bell peper        38
Carrots           38
Cabbage           38
Sauce             38
Salsa.1           37
Cilantro          38
Onion             38
Taquito           38
Pineapple         38
Ham               37
Chile relleno

In [26]:
X_train

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,...,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
77,California,2026-04-25,NaN,NaN,NaN,8.00,4.0,NaN,NaN,21.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,California,2018-01-02,NaN,NaN,NaN,7.25,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,Other,2018-01-09,4.5,3.8,NaN,4.19,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
388,California,2018-01-12,3.5,4.3,NaN,7.00,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,Other,2018-01-12,NaN,NaN,NaN,8.50,4.0,NaN,NaN,21.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,Asada,2018-04-04,NaN,NaN,NaN,7.20,3.0,NaN,NaN,24.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
391,Carnitas,2018-04-09,NaN,NaN,NaN,5.99,3.0,NaN,NaN,20.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,Carnitas,2018-04-09,NaN,NaN,NaN,5.99,3.5,NaN,NaN,17.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,Carnitas,2018-04-09,NaN,NaN,NaN,5.99,2.0,NaN,NaN,19.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
394,California,2018-04-14,NaN,NaN,NaN,8.99,4.0,NaN,NaN,25.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
X_train.describe(exclude='number')

,Burrito,Date,Chips,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,...,Chile relleno,Nopales,Lobster,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
count,38,38,1,3,0,4,7,8,5,3,...,0,0,0,1,0,0,0,0,0,0
unique,5,17,1,1,0,1,1,1,1,1,...,0,0,0,1,0,0,0,0,0,0
top,Other,2019-08-27 00:00:00,No,x,NaN,x,x,x,x,x,...,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
freq,15,9,1,3,NaN,4,7,8,5,3,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,2018-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2026-04-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
columns_with_x = X_train.select_dtypes(exclude='number').drop(['Burrito', 'Date'], axis = 1).columns.to_list()

In [77]:
from sklearn.impute import SimpleImputer

In [88]:
import numpy as np

def convert_x_to_bool(df, imputer:SimpleImputer = None):
    bool_df = df.drop('Queso', axis=1).copy()
    for col in columns_with_x:
        bool_df[col] = bool_df[col].map({'x':True})

    if imputer:
        bool_df[columns_with_x] = pd.DataFrame(imputer.transform(bool_df[columns_with_x]), columns=columns_with_x, index=bool_df.index)
    else:
        imputer = SimpleImputer(strategy="constant", fill_value=False)
        bool_df[columns_with_x] =  pd.DataFrame(
            bool_imputer.fit_transform(bool_df[columns_with_x]),
            columns=columns_with_x,
            index=bool_df.index)

    return bool_df, imputer

In [121]:
X_train_bool, imputer = convert_x_to_bool(X_train)
X_val_bool, imputer = convert_x_to_bool(val['x'])
X_test_bool, imputer = convert_x_to_bool(test['x'])

In [130]:
num_columns = train['x'].select_dtypes(include="number").drop('Queso', axis=1).columns.to_list()

In [135]:
X_train_bool[num_columns].shape
len(num_columns)


18

In [140]:
mean_imputer = SimpleImputer()

In [141]:

len(mean_imputer.fit_transform(X_train_bool[num_columns])[0])



16

In [143]:
len(num_columns)

18

In [119]:
impute_mean(X_train_bool)

ValueError: Columns must be same length as key